# L11: Lettuce Operational Semantics

## Overview
* The Lettuce AST
* Well Formed vs ILL Formed expressions
* Interpreter
* Evaluation
* Teaser on HOFs

## The Lettuce AST

* Here is a grammar that I'm using for lecture today
    * $$\begin{array}{rcll}
\mathbf{Program} & \rightarrow & TopLevel(Expr) \\
\mathbf{Expr} & \rightarrow & Value \\
 & | & Ident(\mathbf{Identifier}) \\
 & | & Plus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Let( \mathbf{Identifier}, \mathbf{Expr}, \mathbf{Expr}) & \text{let identifier = expr in expr} \\
\mathbf{Value} & \rightarrow & Const(\mathbf{Number}) \\
 & | & True \\
 & | & False \\
\mathbf{Identifier} & \rightarrow & String \\
\mathbf{Number} & \rightarrow & Double \\
\end{array}$$

In [1]:
sealed trait Program  // Program
sealed trait Expr  // Expr

case class TopLevel(e1:Expr) extends Program  // TopLevel(Expr)

case class Ident(i1: Identifier) extends Expr  // Ident(Identifier)
case class Plus(e1:Expr, e2:Expr) extends Expr  // Plus(Expr, Expr)
case class Let(i1:Identifier, e1:Expr, e2:Expr) extends Expr  // Let(Identifier, Expr, Expr)

sealed trait Value extends Expr  // Value
case class Const(n:Number) extends Value  // Const(Number)
case object True extends Value  // True
case object False extends Value  // False

type Identifier = String  // Identifier
type Number = Double  // Number

defined trait Program
defined trait Expr
defined class TopLevel
defined class Ident
defined class Plus
defined class Let
defined trait Value
defined class Const
defined object True
defined object False
defined type Identifier
defined type Number

#### Example

~~~
let w = [
        let z = 10 in
            z + z
    ] in w + w
~~~

* Note that here, 'z' is a temp variable defined in the binding expression of 'w'
* The concrete value of the program is: 40

In [3]:
val program1: Program = TopLevel(
    Let(
        "w",
        Let(
            "z",
            Const(10.0),
            Plus(Ident("z"), Ident("z"))
        ),
        Plus(
            Ident("w"),
            Ident("w")
        )
    )
)

val value1:Value = Const(40.0)

// If evaluate was defined, we could assert the following:
// assert(evaluate(program1) == value1)

program1: Program = TopLevel(
  Let(
    "w",
    Let("z", Const(10.0), Plus(Ident("z"), Ident("z"))),
    Plus(Ident("w"), Ident("w"))
  )
)
value1: Value = Const(40.0)

## Well Formed vs ILL Formed expressions
__Fact:__ Abstract Syntax Trees are built from Context Free Grammars. Unfortunately, due to limitations arising
from context freedom (i.e, LHS of each rule is just a single nonterminal), we cannot really enforce rules like 
- Variables should be declared before use 
    - **observed in class last week**
    - We'll review it today today

Consider our definition of the language lettuce. There are some valid expressions that just don't have logical values.

### Lettuce Design Decisions
We shall write our interpreter to catch errors in Lettuce programs at compile time (static time). 
- Variables should be declared before use (we can catch this using a special declare before use check).
    - Catch using the wellFormed function - Scope analysis
    - And it's in the course notes

## wellFormed
* Let us write a function wellFormed that analyzes the scope of variables in our program
* It will ensure that variables are declared before use
* It will ensure that a variable is not used after the scope has ended
* In general, this should help us better understand scope

### Inference Rules Plus
* Operational sematic
    * wellFormed(e, S)
* Plus pattern
    * e1 + e2
* Inference rule
    * $$\begin{array}
???
\\
\hline
???
\\
\end{array} {wellFormedPlus}$$

In [5]:
// operational semantic: wellFormed(e, S)
def wellFormed(e:Expr, S:Set[String] = Set()):Boolean = {
    e match {
        case True | False | Const(_) => true  // wellFormedValue
        case Let(x, eBind, eBody) => wellFormed(eBind,S) && wellFormed(eBody, S + x) // wellFormedLet
        case Ident(x) => S contains x  // wellFormedIdent
        case _ => ???  // wellFormedPlus
        case _ => false
    }
}

defined function wellFormed

#### Testing wellFormed
* Now that we have a wellFormed function, I think we should test it a little bit

* Given the partial implementation of wellFormed that we have so far, what sort of expressions ought to be well formed?

In [6]:
// Concrete: true
val e0: Expr = True
assert(wellFormed(e0))

// Concrete: let x = 1 in x
val e1: Expr = Let("x", Const(1),Ident("x"))
assert(wellFormed(e1))

// Concrete: let x = 2 in x + x
val e2: Expr = ???
assert(wellFormed(e2))

: 

* Given the partial implementation of wellFormed that we have so far, what sort of expressions ought to be ill formed (not well formed)?

In [17]:
// val illFormed:(Expr) => Boolean = (e) => !wellFormed(e)
val illFormed:(Expr) => Boolean = !wellFormed(_)  // _ the magician

// Concrete: x
val e3: Expr = Ident("x")
assert(illFormed(e3))

// Concrete: let x = 1 + x in x + x
val e4: Expr = ???
assert(illFormed(e4))

: 

* What is the wellFormed function? 
    * It is a scope analyzer that runs in static time
    * It checks at static time if the expression is valid
        * static time == compilation time == before evaluation == before dynamic time
    * It states a lot about the scope of a variable/identifier
* See the course notes for more details

## Evaluation of Lettuce
* Let’s write an interpreter for the Lettuce language.

### Operational Semantic (Op. Sem.)
* First we’ll define an operational sematic
    * If I were a programming linguist I would write the op. sem.:
        * $\sigma \models \texttt{e} \Downarrow v$
        * this reads: In a variable environemtn sigma ($\sigma$), the expression “e” is evaluated in 0-or-many steps to a value “v”
    * But I’m not a programming linguist and I don’t love this notation instead, Ill write: “**v = eval(e, env)**”
    * What is env? Why is it useful? We didn’t need this for “Logic” or “Maths”, what could this be?
    * “env” – often denoted $\sigma$ (sigma) - is our variable environment. 
        * It maps Identifiers to values
            * …for now... Later on we’ll allow it to map to locations, expressions and values.
        * environments map variables “x” to values “v”
* Let’s look at the function definition
* Op. Sem.:
    * v = eval(e, env)

In [8]:
// writen as eval0, because, Jupyter Notebooks...
def eval0(e:Expr, env:Map[String, Value] = Map()): Value = ???

defined function eval0

* Observations
    * e: Expr, nothing new there
    * The type of env is “Map[String, Value]”
        * Map[A,B] is a Scala type – often denoted Map[K,V] because key-value pairs...
        * It’s an awful lot like python dictionary, or JavaScript Objects
        * Generally I would call these a variant of an “associative array”
    * The env has a default value "Map()" - the empty map
    * the return type is a “Value” – a subset of all “Expr”
        * Unlike Maths and Logic, Lettuce has many types in it’s value domain
            * Maths returned an Int
            * Logic returned a Boolean
            * eval must return both Doubles and Booleans... but HOW???
                * We have an abstract type, "Value" that works quite well
        * We can no longer return simple Scala types like “Int”, "Double", or “Boolean”
        * We must return an absract type, here "Value"

### Inference Rules
* Now for some inference rules!
* I’ll write a rule for True, and ask you to write your own for False and Const(n)

* True
    * $$\begin{array} & \\
\hline
True\ =\ eval(True,\ env)\\
\end{array} \text{evalTrue}$$

* False
    * $$\begin{array} & 
\\
\hline
False = eval(False, env)
\\
\end{array} \text{evalFalse}$$

* Const
    * $$\begin{array} & 
\\
\hline
Const(n) = eval(Const(n), env) 
\\
\end{array} \text{evalConst}$$

* Let’s code it

In [9]:
def eval1(e:Expr, env:Map[String, Value] = Map()): Value = e match {
    case _ => ???  // evalTrue
    case _ => ???  // evalFalse
    case _ => ???  // evalConst
    case _ => ???
}

defined function eval1

* Values
    * $$\begin{array} & 
\\
\hline
Value = eval(Value, env) \\
\end{array} \text{evalValue}$$

In [10]:
def eval2(e:Expr, env:Map[String, Value] = Map()): Value = e match {
    case _ => ???  // evalVal
    case _ => ???
}

defined function eval2

* Here is an inference rule for Ident
    * $$\begin{array} &
Value = env(Identifier) \\
\hline
Value = eval(Ident(Identifier), env)\\
\end{array} {evalIdent}$$

In [11]:
def eval3(e:Expr, env:Map[String, Value] = Map()): Value = e match {
    case v:Value => v  // evalVal
    case _ => ??? // evalIdent
}

defined function eval3

* Now for some more interesting stuff. Let us consider the let bindings – we’ll start with a stupid statement
    * Concrete Lettuce: **let x = true in false**
        * Lettuce value: **false**
        * Expr (abstract syntax): **Let(“x”, True, False)**
        * Value (abstract syntax): **False**
        * Steps: 
            * eval(let x = true in false, {}) -->
            * eval(false, { “x” -> true } ) --> 
            * false

* A slightly more interesting example
    * Concrete Lettuce: **let x = true in x**
        * Lettuce value: **true**
        * Expr (abstract syntax): **Let(“x”, True, Ident(“x”))**
        * Value (abstract syntax): **True**
        * Steps: 
            * eval(let x = true in x, {}) -->
            * eval(x, { “x” -> true } ) --> 
            * true

* What is the inference rule for let bindings?
    * concrete grammar production: $let\ x\ =\ e_{binding}\ in\ e_{body}$
    * Generative grammar production: $\mathbf{Let(Ident(x), Expr_{binding}, Expr_{body})}$
    * op. sem.: v = eval(e, env)
    * $$ \begin{array} &
v_{binding} = eval(e_{binding}, env)
\hspace{1cm}
env_{prime} = env\ +\ (x\ ->\ v_{binding})
\hspace{1cm}
v = eval(e_{body},\ env_{prime})
\\
\hline
v = eval(\ let\ x\ =\ e_{binding}\ in\ e_{body}\ ,\ env) \\
\end{array} \text{evalLet} $$

In [13]:
def eval4(e:Expr, env:Map[String, Value] = Map()): Value = e match {
    case v:Value => v // evalValue
    case Ident(x) => env(x)  // evalIdent
    case Let(x, e1, e2) => {
        val v1 = eval4(e1, env)
        val envp = env + (x -> v1)
        val v = eval4(e2, envp)
        v
    }  // evalLet
    // case Let(x, e1, e2) => eval(e2, env + (x -> eval(e1, env)))
    case _ => ???
}

println("TESTING")
assert(eval4(Let("x", True, False)) == False)
assert(eval4(Let("dog", True, Ident("dog"))) == True)
println("all tests passed!!!")

TESTING
all tests passed!!!


defined function eval4

### Understanding the code
* consider the assertion
    * assert(eval4(Let("dog", True, Ident("dog"))) == True)
* How does eval4 take Let("dog",True, Ident("dog")) and create True?
* Lets look at a table representation...

#### Table 0
* i is the index of a call to function eval4
* e_i is the value of parameter e during the i<sup>th</sup> call to eval4
* env_i is the value of parameter env during the i<sup>th</sup> call to eval4
* notes_i is a place for us to make notes for the i<sup>th</sup> call to eval4
* value_i is the value of the i<sup>th</sup> call to eval4
* eval4(Let("dog",True, Ident("dog"))
<table>
<thead>
    <tr>
        <th>i</th>
        <th>e_i</th>
        <th>env_i</th>
        <th>notes_i</th>
        <th>value_i</th>
    </tr>
</thead>
<tbody>
    <tr>
        <td>1</td>
        <td>???</td>
        <td>???</td>
        <td>???</td>
        <td>???</td>
    </tr>
    <tr>
        <td>2</td>
        <td>???</td>
        <td>???</td>
        <td>???</td>
        <td>???</td>
    </tr>
    <tr>
        <td>3</td>
        <td>???</td>
        <td>???</td>
        <td>???</td>
        <td>???</td>
    </tr>
    <tr>
        <td>4</td>
        <td>???</td>
        <td>???</td>
        <td>???</td>
        <td>???</td>
    </tr>
</tbody>
</table>

#### Table 1
* eval4(Let("dog",True, Ident("dog")))
<table>
<thead>
    <tr>
        <th>i</th>
        <th>e_i</th>
        <th>env_i</th>
        <th>notes_i</th>
        <th>value_i</th>
    </tr>
</thead>
<tbody>
    <tr>
        <td>1</td>
        <td>Let("dog",True, Ident("dog"))</td>
        <td>{}</td>
        <td>
            <ul>
                <li>We hit the evalLet case with pattern Let(x,e1,e2)</li>
                <ul>
                    <li>x = "dog"</li>
                    <li>e1 = True</li>
                    <li>e2 = Ident("dog")</li>
                </ul>
                <li>We find v1 = eval4(e1,env)</li>
                <li>We find envp = env + (x -> v1)</li>
                <li>We return eval4(e2, envp)</li>
            </ul>
        </td>
        <td>???</td>
    </tr>
    <tr>
        <td>2</td>
        <td>???</td>
        <td>???</td>
        <td>???</td>
        <td>???</td>
    </tr>
    <tr>
        <td>3</td>
        <td>???</td>
        <td>???</td>
        <td>???</td>
        <td>???</td>
    </tr>
</tbody>
</table>

#### Table 2
* Note changes to row 2
* eval4(Let("dog",True, Ident("dog")))
<table>
<thead>
    <tr>
        <th>i</th>
        <th>e_i</th>
        <th>env_i</th>
        <th>notes_i</th>
        <th>value_i</th>
    </tr>
</thead>
<tbody>
    <tr>
        <td>1</td>
        <td>Let("dog",True, Ident("dog"))</td>
        <td>{}</td>
        <td>
            <ul>
                <li>We hit the evalLet case with pattern Let(x,e1,e2)</li>
                <ul>
                    <li>x = "dog"</li>
                    <li>e1 = True</li>
                    <li>e2 = Ident("dog")</li>
                </ul>
                <li>We find v1 = eval4(e1,env)</li>
                <li>We find envp = env + (x -> v1)</li>
                <li>We return eval4(e2, envp)</li>
            </ul>
        </td>
        <td>???</td>
    </tr>
    <tr>
        <td>2</td>
        <td>True</td>
        <td>{}</td>
        <td>
            <ul>
                <li>We hit the evalValue case with pattern True</li>
                <li>return the pattern</li>
            </ul>
        </td>
        <td>True</td>
    </tr>
    <tr>
        <td>3</td>
        <td>???</td>
        <td>???</td>
        <td>???</td>
        <td>???</td>
    </tr>
</tbody>
</table>

#### Table 3
* Note changes to notes_1
* eval4(Let("dog",True, Ident("dog")))
<table>
<thead>
    <tr>
        <th>i</th>
        <th>e_i</th>
        <th>env_i</th>
        <th>notes_i</th>
        <th>value_i</th>
    </tr>
</thead>
<tbody>
    <tr>
        <td>1</td>
        <td>Let("dog",True, Ident("dog"))</td>
        <td>{}</td>
        <td>
            <ul>
                <li>We hit the evalLet case with pattern Let(x,e1,e2)</li>
                <ul>
                    <li>x = "dog"</li>
                    <li>e1 = True</li>
                    <li>e2 = Ident("dog")</li>
                </ul>
                <li>We find v1 = eval4(e1,env)</li>
                <ul>
                    <li>We've found that v1 = True</li>
                </ul>
                <li>We find envp = env + (x -> v1)</li>
                <ul>
                    <li>We've found that v1 = True</li>
                    <li>envp = env + (x -> v1)</li>
                    <li>envp = {} + ("dog" -> True)</li>
                    <li>envp = {"dog" -> True}</li>
                </ul>
                <li>We return eval4(e2, envp)</li>
            </ul>
        </td>
        <td>???</td>
    </tr>
    <tr>
        <td>2</td>
        <td>True</td>
        <td>{}</td>
        <td>
            <ul>
                <li>We hit the evalValue case with pattern True</li>
                <li>return the pattern</li>
            </ul>
        </td>
        <td>True</td>
    </tr>
    <tr>
        <td>3</td>
        <td>???</td>
        <td>???</td>
        <td>???</td>
        <td>???</td>
    </tr>
</tbody>
</table>

#### Table 4
* Note changes in last row
* eval4(Let("dog",True, Ident("dog")))
<table>
<thead>
    <tr>
        <th>i</th>
        <th>e_i</th>
        <th>env_i</th>
        <th>notes_i</th>
        <th>value_i</th>
    </tr>
</thead>
<tbody>
    <tr>
        <td>1</td>
        <td>Let("dog",True, Ident("dog"))</td>
        <td>{}</td>
        <td>
            <ul>
                <li>We hit the evalLet case with pattern Let(x,e1,e2)</li>
                <ul>
                    <li>x = "dog"</li>
                    <li>e1 = True</li>
                    <li>e2 = Ident("dog")</li>
                </ul>
                <li>We find v1 = eval4(e1,env)</li>
                <ul>
                    <li>We've found that v1 = True</li>
                </ul>
                <li>We find envp = env + (x -> v1)</li>
                <ul>
                    <li>We've found that v1 = True</li>
                    <li>envp = env + (x -> v1)</li>
                    <li>envp = {} + ("dog" -> True)</li>
                    <li>envp = {"dog" -> True}</li>
                </ul>
                <li>We return eval4(e2, envp)</li>
            </ul>
        </td>
        <td>???</td>
    </tr>
    <tr>
        <td>2</td>
        <td>True</td>
        <td>{}</td>
        <td>
            <ul>
                <li>We hit the evalValue case with pattern True</li>
                <li>return the pattern</li>
            </ul>
        </td>
        <td>True</td>
    </tr>
    <tr>
        <td>3</td>
        <td>Ident("dog")</td>
        <td>{"dog" -> True}</td>
        <td>
            <ul>
                <li>We hit the evalIdent case with pattern Ident(x)</li>
                <ul>
                    <li>Here, x is "dog"</li>
                </ul>
                <li>We return env(x)</li>
                <li>env(x) == {"dog" -> True}("dog") == True</li>
            </ul>
        </td>
        <td>True</td>
    </tr>
</tbody>
</table>

#### Table 5
* Note change to return value call 1
* eval4(Let("dog",True, Ident("dog")))
<table>
<thead>
    <tr>
        <th>i</th>
        <th>e_i</th>
        <th>env_i</th>
        <th>notes_i</th>
        <th>value_i</th>
    </tr>
</thead>
<tbody>
    <tr>
        <td>1</td>
        <td>Let("dog",True, Ident("dog"))</td>
        <td>{}</td>
        <td>
            <ul>
                <li>We hit the evalLet case with pattern Let(x,e1,e2)</li>
                <ul>
                    <li>x = "dog"</li>
                    <li>e1 = True</li>
                    <li>e2 = Ident("dog")</li>
                </ul>
                <li>We find v1 = eval4(e1,env)</li>
                <ul>
                    <li>We've found that v1 = True</li>
                </ul>
                <li>We find envp = env + (x -> v1)</li>
                <ul>
                    <li>We've found that v1 = True</li>
                    <li>envp = env + (x -> v1)</li>
                    <li>envp = {} + ("dog" -> True)</li>
                    <li>envp = {"dog" -> True}</li>
                </ul>
                <li>We return eval4(e2, envp)</li>
            </ul>
        </td>
        <td>True</td>
    </tr>
    <tr>
        <td>2</td>
        <td>True</td>
        <td>{}</td>
        <td>
            <ul>
                <li>We hit the evalValue case with pattern True</li>
                <li>return the pattern</li>
            </ul>
        </td>
        <td>True</td>
    </tr>
    <tr>
        <td>3</td>
        <td>Ident("dog")</td>
        <td>{"dog" -> True}</td>
        <td>
            <ul>
                <li>We hit the evalIdent case with pattern Ident(x)</li>
                <ul>
                    <li>Here, x is "dog"</li>
                </ul>
                <li>We return env(x)</li>
                <li>env(x) == {"dog" -> True}("dog") == True</li>
            </ul>
        </td>
        <td>True</td>
    </tr>
</tbody>
</table>

## Evaluation
* Until now, I've sort of ignored the type Program
* And we haven't seen how **wellFormed** and **eval** play together
* Here is the gist

In [14]:
def evaluate(p:Program):Value = {
    val TopLevel(e) = p
    if (wellFormed(e)) {
        eval4(e)
    } else {
        ???  // throw some sort of error
    }
}

defined function evaluate

* wellFormed is executed before evaluation - in static time
    * static == compilation == before evaluation
* Then eval is executed in dynamic time
    * dynamic == during evaluation ~~ state dependent

### Example

* What would happen evaluating this expression:
~~~
let dog = cat in fish
~~~

* What would happen evaluating this expression:
~~~
let dog = 1 + 2 in dog + dog
~~~

* What would happen evaluating this expression:
~~~
let dog = 5 + dog in dog
~~~

* What would happen evaluating this expression:
~~~
let dog = [
    let cat = 1 + 2 in cat + cat
] in dog + dog
~~~

* What would happen evaluating this expression:
~~~
let dog = [
    let cat = 1 + 2 in cat + cat
] in dog + cat
~~~

## Teaser HOF
* Until now in this class I’ve only really talked about “First class functions”
* But there are also “Higher ordered functions” (HOFs)
    * If a function returns a function as output, then it is an HOF
    * If a function takes another function as input, then it is an HOF
* In homework 3 you implemented an HOF named filterByN for a NumList defined something like:
    * $$\begin{array}
\mathbf{NumList} & \rightarrow & \mathbf{EmptyList} \\
& | &  \mathbf{Cons(Double, NumList)} \\
\end{array}$$
* Let’s look at this further, see code bellow:


In [4]:
// Look at some tests for filterByN
// Implement filterByN
// Implement a method called filter for NumList - don't use the visitors pattern
// Write Tests


// sealed trait NumList {
//     // Note, no need to use the visitors pattern for this!
//     def filter(f:(Double) => Boolean):NumList = this match {
//         case EmptyList => ???
//         case Cons(h,t) => ???
//     }
// }

sealed trait NumList  // NumList
case object EmptyList extends NumList  // EmptyList
case class Cons(head:Double, tail:NumList) extends NumList  // Cons(Double, NumList)


def filterByN(lst:NumList, f:(Double) => Boolean):NumList = {
    ???
}


//
// TESTS
//
val l0 = EmptyList  // []
val l1 = Cons(1, Cons(2, Cons(3, EmptyList)))  // [1,2,3]
val l2 = Cons(16, Cons(28, Cons(8, EmptyList)))  // [16, 28, 8]

val evenL0 = EmptyList  // []
val evenL1 = Cons(2, EmptyList)  // [2]
val evenL2 = Cons(16, Cons(28, Cons(8, EmptyList)))  // [16, 28, 8]

val oddL0 = EmptyList  // []
val oddL1 = Cons(1, Cons(3, EmptyList))  // [1,3]
val oddL2 = EmptyList  // []

val isEven = (x:Double) => { x % 2 == 0 }
val isOdd = (x:Double) => { x % 2 == 1 }

println("Testing filterByN")

    assert(filterByN(l0, isEven) == evenL0)
    assert(filterByN(l1, isEven) == evenL1)
    assert(filterByN(l2, isEven) == evenL2)

    // Don't need the isEven val...
    assert(filterByN(l2, (x:Double) => { x % 2 == 0 }) == evenL2)
    assert(filterByN(l2, { _ % 2 == 0 }) == evenL2)

    assert(filterByN(l0,isOdd) == oddL0)
    assert(filterByN(l1,isOdd) == oddL1)
    assert(filterByN(l2,isOdd) == oddL2)
    assert(filterByN(l2,{_%2 == 1}) == oddL2)

println("PASSED filterByN!!!")


// println("Testing filter")
//     assert((l0 filter isEven) == evenL0)
//     assert((l1 filter isEven) == evenL1)
//     assert((l2 filter isEven) == evenL2)
//     assert((l0 filter isOdd) == oddL0)
//     assert((l1 filter isOdd) == oddL1)
//     assert((l2 filter isOdd) == oddL2)
// println("PASSED filter!!!")

Testing filterByN


: 

### Scala List[A]
#### filter
* Scala has a type List[A]
* One of the methods of List[A] is a method called filter
* filter might looks something like bellow:

~~~
sealed trait List[+A] {
	def filter[A](f:(A) => Boolean):List[A] = this match {
		case Nil => Nil
		case head :: tail => {
			if ( f(head) ) {
				head :: (tail filter f)
			} else {
				tail filter f
			}
		}
	}
}
~~~

* Note that this works on an abstract type “A”
    * It’s nothing special
    * It exists in other languages as well

* Here is how I can use it:




In [ ]:
// here A is a double...
val lDoubles = List(1.0,2.0,3.0,4.0,5.0,6.0)
val lDoublesRelEven3:List[Double] = List(3.0,6.0)
val isRelEven3 = (d:Double) => (d%3) == 0
assert((lDoubles filter isRelEven3) == lDoublesRelEven3)

In [ ]:
// here A is Char
val lChars = List('a','b','c','d','e','f','g')
val lVowels = List('a','e')
val lConst = List('b','c','d', 'f','g')
val isVowel = (c:Char) => Set('a','e','i','o','u') contains c
assert((lChars filter isVowel) == lVowels)
assert((lChars filter {!isVowel(_)}) == lConst)  // _ the all powerfull!

#### map
* Scala’s type List[A] also has a method map, it might look like bellow:

~~~
sealed trait List[+A] {
	def map[A,B](f:(A) => B):List[B] = this match {
		case Nil => Nil
		case head :: tail => f(head) :: (tail map f)
	}
}
~~~

* Here is how I can use it:




In [ ]:
// here A is an Int
// here B is a Boolean
val lInts = List(5,8,13)
println(lInts map {(n:Int) => n % 2 == 0})
assert((lInts map {(n:Int) => n % 2 == 0}) == List(false, true, false))
assert((lInts map {_ % 2 == 0}) == List(false, true, false))

In [ ]:
// here A is an Int
// here B is an Int
println(lInts map {(n:Int) => n + 1})
assert((lInts map {(n:Int) => n + 1}) == List(6,9,14))
assert((lInts map {_ + 1}) == List(6,9,14))

#### foldLeft
* Scala’s type List[A] also has a method foldLeft
* A better name might be “scanFromLeftToRightAndAccumulateSomeValue”
* But then I doubt anyone would use it…
* it might look like bellow:
~~~
sealed trait List[+A] {
	def foldLeft[A,B](acc:B)(f:(B, A) => B):B = this match {
		case Nil => acc
		case head :: tail => {
			val accPrime = f(acc, head)
			tail.foldLeft(accPrime)(f)
		}
	}
}
~~~

* Here is how I can use it:




In [ ]:
// Here A is an Int and B is also an Int
val sum = List(1,2,3).foldLeft(0){
    (acc, h) => {
        println(s"found list element $h")
        println(s"\tacc is: $acc")
        acc + h
    }
}

println(s"sum is: $sum")
assert(List(1,2,3).foldLeft(0){_ + _} == 6)  // #LoDashCan'tCompete

In [ ]:
// Here A is Int and B is String
def formatList(l:List[Int]):String = {
    l.foldLeft(""){
        case ("", h) => s"[ $h"
        case (acc, h) => s"$acc, $h"
    } + " ]"
}

println(formatList(List(1,2,3)))

## Overview
* The Lettuce AST
* Well Formed vs ILL Formed expressions
* Interpreter
* Evaluation
* Teaser on HOFs

## TODOs
* Homework and Quiz 4 is due Friday 02/22
* Homework 5 is due Sunday 02/24 - No Moodle Quiz!
* Project 1 is due Monday 03/04
* If you have questions about your spot exam, please let us know - in person or over Piazza
    * Some time in Recitation this Friday should be dedicated to reviewing the first spot exam.